#Data Load

In [1]:
!pip install lightgbm


[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
!pip install lightgbm==3.3.2

     ---------------------------------------- 1.0/1.0 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.3.0
    Uninstalling lightgbm-4.3.0:
      Successfully uninstalled lightgbm-4.3.0

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\~ightgbm\\bin\\lib_lightgbm.dll'
Consider using the `--user` option or check the permissions.



In [1]:
import lightgbm
print(lightgbm.__version__)

3.3.2


In [2]:
import warnings
warnings.filterwarnings('ignore')

# 라이브러리

import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [2]:
#-*- coding:utf-8 -*-

In [3]:
!python --version
print('pandas', pd.__version__)
print('numpy', np.__version__)

Python 3.9.7
pandas 1.3.4
numpy 1.18.5


In [4]:
# Seed 고정

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) 

In [5]:
train = pd.read_csv('../train.csv',encoding = 'utf-8')
test = pd.read_csv('../test.csv',encoding = 'utf-8')
sub = pd.read_csv('../sample_submission.csv',encoding = 'utf-8')

# Feature Engineering

> 라벨인코딩

In [6]:
# qualitative to quantitative

qual_col = ['PRODUCT_CODE', 'LINE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')

Done.


In [7]:
train_x = train.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [8]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> 정규화

In [9]:
a=train.columns.str.contains('X')
print(a)


[False False False ...  True  True  True]


In [10]:
#.columns.str.contains("a") : a라는 문자가 들어있는 문자열 모두 찾음
x_col = train.columns[train.columns.str.contains('X')].tolist()
x_col

['X_1',
 'X_2',
 'X_3',
 'X_4',
 'X_5',
 'X_6',
 'X_7',
 'X_8',
 'X_9',
 'X_10',
 'X_11',
 'X_12',
 'X_13',
 'X_14',
 'X_15',
 'X_16',
 'X_17',
 'X_18',
 'X_19',
 'X_20',
 'X_21',
 'X_22',
 'X_23',
 'X_24',
 'X_25',
 'X_26',
 'X_27',
 'X_28',
 'X_29',
 'X_30',
 'X_31',
 'X_32',
 'X_33',
 'X_34',
 'X_35',
 'X_36',
 'X_37',
 'X_38',
 'X_39',
 'X_40',
 'X_41',
 'X_42',
 'X_43',
 'X_44',
 'X_45',
 'X_46',
 'X_47',
 'X_48',
 'X_49',
 'X_50',
 'X_51',
 'X_52',
 'X_53',
 'X_54',
 'X_55',
 'X_56',
 'X_57',
 'X_58',
 'X_59',
 'X_60',
 'X_61',
 'X_62',
 'X_63',
 'X_64',
 'X_65',
 'X_66',
 'X_67',
 'X_68',
 'X_69',
 'X_70',
 'X_71',
 'X_72',
 'X_73',
 'X_74',
 'X_75',
 'X_76',
 'X_77',
 'X_78',
 'X_79',
 'X_80',
 'X_81',
 'X_82',
 'X_83',
 'X_84',
 'X_85',
 'X_86',
 'X_87',
 'X_88',
 'X_89',
 'X_90',
 'X_91',
 'X_92',
 'X_93',
 'X_94',
 'X_95',
 'X_96',
 'X_97',
 'X_98',
 'X_99',
 'X_100',
 'X_101',
 'X_102',
 'X_103',
 'X_104',
 'X_105',
 'X_106',
 'X_107',
 'X_108',
 'X_109',
 'X_110',
 'X_111'

In [11]:
train_x[x_col]

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from sklearn.preprocessing import MinMaxScaler

x_col = train.columns[train.columns.str.contains('X')].tolist()
scaler = MinMaxScaler()
  
scaler.fit(train_x[x_col])
  
train_x[x_col] = scaler.transform(train_x[x_col])
test_x[x_col] = scaler.transform(test_x[x_col])
  
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


> 결측값 'train_x' 평균으로 채우기

In [13]:
train_x = train_x.fillna(train_x.mean())# mean 연산시 nan값은 분석 대상에서 제외
test_x = test_x.fillna(train_x.mean())

In [14]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.256757,0.248647,0.000000,0.122283,0.890487,0.0,NaN,NaN,NaN,NaN
1,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.240754,0.300866,0.407899,0.164742,0.601770,0.0,NaN,NaN,NaN,NaN
2,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.251422,0.133929,0.355835,0.205163,0.922566,0.0,NaN,NaN,NaN,NaN
3,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.199858,0.202110,0.704129,0.003057,0.559181,0.0,NaN,NaN,NaN,NaN
4,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.233997,0.275703,0.515978,0.088315,0.846239,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,0.009804,0.533333,0.0,0.0,0.00000,0.0,0.294118,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
594,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.616999,0.578193,0.658169,0.835938,0.266593,0.0,NaN,NaN,NaN,NaN
595,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
596,4,1,0.382353,0.466667,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN


> 전체 결측값(Null)인 X변수 Drop하기

In [15]:
# 결측값있는 열 전체 삭제
train_x = train_x.dropna(axis =1)
test_x = test_x.dropna(axis=1)

In [16]:
# 결측값 확인
train_x.isnull()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
594,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
595,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
596,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
train_x.isnull().sum()

LINE            0
PRODUCT_CODE    0
X_1             0
X_2             0
X_3             0
               ..
X_2867          0
X_2868          0
X_2869          0
X_2870          0
X_2871          0
Length: 2795, dtype: int64

In [18]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.918723,0.100000,0.113966,0.119565,0.256757,0.248647,0.000000,0.122283,0.890487,0.0
1,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.869376,0.100000,0.107939,0.119565,0.240754,0.300866,0.407899,0.164742,0.601770,0.0
2,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.577649,0.100000,0.103476,0.119565,0.251422,0.133929,0.355835,0.205163,0.922566,0.0
3,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.582003,0.110000,0.123562,0.119565,0.199858,0.202110,0.704129,0.003057,0.559181,0.0
4,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.825835,0.100000,0.103810,0.108696,0.233997,0.275703,0.515978,0.088315,0.846239,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,0.009804,0.533333,0.0,0.0,0.00000,0.0,0.294118,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
594,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.624093,0.110000,0.130258,0.119565,0.616999,0.578193,0.658169,0.835938,0.266593,0.0
595,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.448476,0.100000,0.100229,0.108696,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
596,4,1,0.382353,0.466667,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0


In [19]:
train_y

0      1
1      2
2      1
3      2
4      1
      ..
593    1
594    0
595    0
596    1
597    1
Name: Y_Class, Length: 598, dtype: int64

In [20]:
test_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,5,2,0.009804,0.466667,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
1,4,2,0.009804,0.400000,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
2,4,2,0.009804,0.533333,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
3,0,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.843251,0.940000,0.930103,0.880435,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
4,1,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.783745,0.920000,0.919557,0.880435,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,5,2,0.009804,0.266667,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
306,4,2,0.009804,0.600000,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
307,5,2,0.009804,0.266667,0.0,0.0,0.00000,0.0,0.294118,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
308,5,2,0.009804,0.533333,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0


# Modeling

In [21]:
train_x.shape, train_y.shape, test_x.shape

((598, 2795), (598,), (310, 2795))

In [ ]:
#lightGBM, ExtraTreesClassifier, RandomForestClassifier 선택

#light GBM : 트리기반의 학습 알고리즘인 gradient boosting 방식 프레임워크
# 특징: 다른 알고리즘이 트리를 수평으로 확장하는 것에 반해 트리를 수직으로 확장.
# 일반 level-wise(수평확장) 보다 낮은 loss 달성 경향있음
# 데이터의 크기가 작은 경우 leaf-wise(수직확장)는 과적합이 되기 쉬우므로 max_depth 줄여줘야함
# Light GBM의 'Light'에 걸맞게 속도가 빠른 것이 굉장히 큰 장점이다. 
# 또한, 메모리를 적게 차지하고 속도가 빠르다는 장점 외에도, LGBM은 결과의 정확도가 높다는 장점이 있고 GPU를 활용할 수 있기 때문에 널리 사용되고 있다.

model = lgbm.LGBMClassifier()
et_cls = ExtraTreesClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
rf_cls = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
lg_cls = model.fit(train_x, train_y)
# n_estimators : 생성할 트리의 개수(default = 100)
# min_samples_leaf : 노드를 분할하기 위해 리프 노드가 가져야 할 최소한의 샘플 개수 
# -> 값을 작게 설정하면 모델 복잡성 증가:과적합 위험 값을 증가 시키면 모델의 편향은 증가하고 분산은 감소. 모델의 복잡도가 낮아지지만 일부 손실 발생
# min_samples_split : 노드를 분할 하기 위해 필요한 최소한의 샘플 개수
# -> 샘플 수가 이보다 작아지면 분할하지 않고 리프 노드로 설정 // 값이 작을 수록 복잡성 증가. 데이터 작은 변동에 민감반응 과적합위험.
# max_features : 노드를 분할 할 때 고려 할 특성의 무작위 하위 집합의 크기
# -> 값이 작으면 분산 감소, 편향증가, 복잡도 감소 , 모델 성능 감소
# best_estimator : gridsearchCV 객체에서 최적의 추정기(estimator)를 반환하는 속성
# feaure_importances_ : 랜덤포레스트는 다수의 의사결정 트리 앙상블하여 모델 구성. 각 피처들을 사용하여 분할 수행하고 이 과정에서 피처들의 중요도 계산
# max_depth = 트리의 최대 깊이(defualt = none) : 깊이가 제한된 트리는 각각의 결정 경계를 더 간단하게 만듬. 모델 과적합 방지
# -> 값이 작으면 모델의 단순성을 증가. 분산감소 편향 증가 => 데이터의 다양한 패턴 잡아내지 못하고 일반화 능력 향상됨

# 모델 voting

voting = VotingClassifier(
    estimators=[
        ('et', et_cls),
        ('rf', rf_cls),
        ('lg', lg_cls)
    ]
)
voting.fit(train_x, train_y)

## RandomForestClassifier

In [21]:
import time
import datetime # datetime 라이브러리 import
from sklearn.model_selection import GridSearchCV
start = time.time() # 시작
params = {
    'n_estimators':[500, 1000, 1500, 2000, 2500, 3300],
    'max_depth' : [5, 6, 8, 10, 12], 
    'min_samples_leaf' : [5, 6, 7, 8, 9],
    'min_samples_split' : [6, 7, 8, 9, 10, 12]
}
rf_cls = RandomForestClassifier(random_state = 37, n_jobs = 1)#n_jobs : 모델 학습 및 예측에 사용되는 병렬 작업의 수 -1의 경우 cpu의 모든 코어 사용
grid_cv = GridSearchCV(rf_cls, param_grid = params, cv = 2, scoring = 'accuracy',n_jobs=1, verbose =2 )
grid_cv.fit(train_x, train_y)

# 최적의 파라미터 모델을 이용하여 예측값 생성
estimator = grid_cv.best_estimator_
pred = estimator.predict(test_x)
# 후보 파라미터 성능 검증 결과 출력
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
sec = time.time()-start # 종료 - 시작 (걸린 시간)
times = str(datetime.timedelta(seconds=sec))
short = times.split(".")[0] # 초 단위 까지만
print(f"{times} sec")
print(f"{short} sec")
# cv는 참고용으로만 하는게 일반적이다 꼭 이를 활용하고 싶다면 Out Of Fold prediction으로 이를 활용 할수있다.
# cv로 파라미터 튜닝을 하는 이유는 test셋가 train셋의 일정 부분 다를 수 있기 때문에 여러번 cv해서 평균적인 결과를 반영한 최적의 파라미터를 결정하는 방식.
# cv를 했다고 해서 반드시 test 세트로 평가한 결과와 유사하다고 할 수 없다.
# OOF는 CV로 지정된 FOLD 세트 별로 iteration(평가) 수행하면서 각각 학습한다. 이렇게 개별 학습된 모델 별로 테스트 데이터 셋을 예측하고 이러한 결과를 평균내어 최종 예측함
# KFold 생성(CV=5)
# 5번 KFold iteration 수행 시작
#         학습 데이터와 검증 데이터 분리
#         분리된 학습 데이터와 검증 데이터를 기반으로 Estimator 학습 수행
#         학습된 Estimator를 기반으로 테스트 데이터에 예측
#         예측 결과값을 더한 후 각각 평균 적용.

Fitting 2 folds for each of 900 candidates, totalling 1800 fits
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1000; total time=   1.8s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1000; total time=   1.8s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1500; total time=   2.7s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1500; total time=   2.7s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=2000; total time=   3.5s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=2000; total time=   3.9s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=2500; total time=   4.7s
[CV] END max_depth=5, min_sam

[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=6, n_estimators=2500; total time=   4.6s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=6, n_estimators=3300; total time=   6.0s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=6, n_estimators=3300; total time=   6.2s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1000; total time=   1.8s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1000; total time=   1.8s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1500; total time=   2.7s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1500; total time=   2.8s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=2000; total time=

[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=2000; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=2500; total time=   4.4s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=2500; total time=   4.5s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=3300; total time=   6.1s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=3300; total time=   6.9s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=1000; total time=   1.9s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=1000; total time=   1.8s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=1500; total time=

[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=1500; total time=   3.2s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2000; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2000; total time=   4.4s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2500; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2500; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=3300; total time=   6.4s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=3300; total time=   7.0s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=9, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=9, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=9, n_estimators=1000; total time=

[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=1000; total time=   1.8s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=1500; total time=   2.7s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=1500; total time=   2.9s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2000; total time=   3.6s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2000; total time=   3.7s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2500; total time=   4.5s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2500; total time=   4.8s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=3300; total time=   6.0s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=3300; total time=   6.2s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=10, n_estimators=500; total tim

...

[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=500; total time=   1.2s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1000; total time=   2.5s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1000; total time=   2.6s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1500; total time=   3.8s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1500; total time=   4.0s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=2000; total time=   5.1s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=2000; total time=   4.7s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=2500; total time=   6.0s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=2500; t

[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=8, n_estimators=2500; total time=   5.7s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=8, n_estimators=3300; total time=   7.5s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=8, n_estimators=3300; total time=   8.1s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=1000; total time=   2.2s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=1000; total time=   2.2s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=1500; total time=   3.3s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=1500; total time=   3.3s
[CV] END max_depth=12, min_samples_leaf=6, min_samples_split=9, n_estimators=2000; t

[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=9, n_estimators=2000; total time=   4.0s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=9, n_estimators=2000; total time=   4.1s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=9, n_estimators=2500; total time=   5.1s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=9, n_estimators=2500; total time=   5.1s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=9, n_estimators=3300; total time=   6.7s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=9, n_estimators=3300; total time=   7.3s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=10, n_estimators=500; total time=   1.2s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=10, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=10, n_estimators=1000; total time=   2.2s
[CV] END max_depth=12, min_samples_leaf=7, min_samples_split=10, n_estimators=100

[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=1000; total time=   2.1s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=1500; total time=   3.0s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=1500; total time=   3.2s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=2000; total time=   4.1s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=2000; total time=   4.1s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=2500; total time=   5.2s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=2500; total time=   5.2s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=3300; total time=   7.0s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=10, n_estimators=3300; total time=   7.0s
[CV] END max_depth=12, min_samples_leaf=8, min_samples_split=12, n_estima

[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=500; total time=   1.2s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=1000; total time=   2.5s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=1000; total time=   2.2s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=1500; total time=   3.4s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=1500; total time=   3.3s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=2000; total time=   4.1s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=2000; total time=   4.4s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimators=2500; total time=   5.3s
[CV] END max_depth=12, min_samples_leaf=9, min_samples_split=12, n_estimato

## OOF(Out Of Fold) prediction & LGBMClassifier

In [22]:
train_x.shape[0]
test_x.shape[0]

310

In [23]:
from sklearn.model_selection import KFold
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss

In [25]:
#검증데이터 학습데이터 분리
cut = int(len(train_x)*0.8)
train_xx = train_x[:cut]# 앞에서부터 80퍼까지
valid_xx = train_x[cut:]# 80퍼이후 20퍼 데이터
train_yy = train_y[:cut]
valid_yy = train_y[cut:]
print(train_xx.shape, train_yy.shape, valid_xx.shape, valid_yy.shape)

(478, 2795) (478,) (120, 2795) (120,)


In [29]:
valid_yy.unique()

array([0, 1, 2], dtype=int64)

In [36]:
t_param = {
        'objective': 'multiclass', # 분류
        'verbose': -1, #학습 출력 결과 n_estimators 실행회수 100번마다 결과 출력
        'metric': 'multi_logloss',
        'n_estimators': 500
}
t_model_lgb = lgb.LGBMClassifier(**t_param)## *arg는 튜플형식가변인자, **kwargs 딕셔너리 키워드 
t_lgb_model = t_model_lgb.fit(train_xx, train_yy, eval_set =[(valid_xx, valid_yy)], verbose = False, early_stopping_rounds = 100)#eval_set 검증데이터 셋 지정, early_stoping_rounds : 검증데이터n개를 학습기를 통해 가중치 부여하며 오류개선 -> n_estimators횟수반복 ->loss나 성능지표가 25회 동안 향상되지 않으면 조기종료
t_lgb_pred = t_lgb_model.predict_proba(valid_xx)
# log_score = log_loss(valid_yy,lgb_pred) # 분류문제의 대표적인 평가지표 교차 엔트로피라고 부르기도함. 실제 값을 예측하는 확률에 로그를 취하여 부호를 반전 시킨값. 낮을수록 좋은 지표

In [33]:
np.unique(t_lgb_pred)

array([0, 1, 2], dtype=int64)

In [37]:
t_lgb_pred

array([[0.21039111, 0.68469744, 0.10491145],
       [0.08421394, 0.50544435, 0.4103417 ],
       [0.0949629 , 0.77075056, 0.13428654],
       [0.06954856, 0.79452229, 0.13592915],
       [0.05744625, 0.84137852, 0.10117524],
       [0.1047054 , 0.80590889, 0.08938571],
       [0.07513811, 0.83203256, 0.09282933],
       [0.05647357, 0.85273999, 0.09078644],
       [0.0949696 , 0.82222155, 0.08280885],
       [0.11353303, 0.72370943, 0.16275754],
       [0.09834948, 0.81047926, 0.09117127],
       [0.07766027, 0.82232069, 0.10001904],
       [0.07494343, 0.8098011 , 0.11525547],
       [0.10123237, 0.75899471, 0.13977293],
       [0.08797844, 0.84286931, 0.06915225],
       [0.0591573 , 0.84013847, 0.10070424],
       [0.0634833 , 0.83801884, 0.09849786],
       [0.11755113, 0.74822258, 0.13422629],
       [0.06252121, 0.86211619, 0.0753626 ],
       [0.06680948, 0.84307365, 0.09011687],
       [0.1025094 , 0.81788199, 0.07960862],
       [0.07266604, 0.84696132, 0.08037265],
       [0.

In [38]:
# OOF하기전에 optuna 사용해서 최적 하이퍼파라미터 찾기
from optuna.samplers import TPESampler# objective와 study를 정의하고, n_trails 파라미터를 조정하여 몇 회의 trial 수행할지 설정하는 방식
sampler = TPESampler(seed = 10)
def objective(trial):#trial : study 내의 optimization 단일 수행 // objective는 매 trial을 input으로 받는 함수
    dtrain = lgb.Dataset(train_xx, label = train_yy)
    dtest = lgb.Dataset(valid_xx, label = valid_yy)
    param = {
        'objective': 'multiclass', # 분류
        'verbose': -1, #학습 출력 결과 n_estimators 실행회수 100번마다 결과 출력
        'metric': 'multi_logloss', #평가지표
#         'metric': 'rmse', # 회귀모델 평가지표 
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5), #L1 Regularization parameter로 불필요한 가중치를 0으로 만든다.
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),#L2 Regularization parameter로 불필요한 가중치를 0으로 만든다.
        "num_leaves": trial.suggest_int("num_leaves", 2, 256), #tree의 최대 leaf node의 개수이다.
        'max_depth': trial.suggest_int('max_depth',3, 15), #suggest_int 범위 내의 정수형 값 선택 // 깊이 제한 하는 파라미터 깊이제한이 클수록 과적합이 발생하는 것이
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2), #suggest_loguniform 범위 내의 로그 함수 선상값 선택// 학습률을 의미
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000), # 반복훈련 하려는 트리 개수 지정
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100), # 주요 하이퍼파라미터, 큰 값으로 설정하면 트리가 깊이지는것을 방지 // leaf node에 포함 되어야 할 최소한의 데이터 개수 의미
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),# boosting 단계마다 데이터를 랜덤으로 선택하는 비율. 즉 subsample 비율 만큼 데이터를 랜덤으로 선택해서 학습 진행// 값이 작으면 속도를 향상 할수 잇고 overfitting방지
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.4, 1.0),#boosting 단계마다 feature를 선택하는 비율이다. feature가 많거나 소수의 feature에 의존적일때 overfitting해결 할 수 있다.
    }
    
    model_lgb = lgb.LGBMClassifier(**param)## *arg는 튜플형식가변인자, **kwargs 딕셔너리 키워드 
    lgb_model = model_lgb.fit(train_xx, train_yy, eval_set =[(valid_xx, valid_yy)], verbose = False, early_stopping_rounds = 100)#eval_set 검증데이터 셋 지정, early_stoping_rounds : 검증데이터n개를 학습기를 통해 가중치 부여하며 오류개선 -> n_estimators횟수반복 ->loss나 성능지표가 25회 동안 향상되지 않으면 조기종료
    lgb_pred = lgb_model.predict_proba(valid_xx)
    #rmse = RMSE(valid_yy, lgb_pred) : 평균제곱오차 회귀 예측 모델로 예측값과 실제값 사이의 평균 측정
    log_score = log_loss(valid_yy,lgb_pred) # 분류문제의 대표적인 평가지표 교차 엔트로피라고 부르기도함. 실제 값을 예측하는 확률에 로그를 취하여 부호를 반전 시킨값. 낮을수록 좋은 지표
    return log_score
#study_name='lgbm_papameter_opt',
study_lgb = optuna.create_study(direction='minimize', sampler=sampler) #>> study 정의 minimize: logloss를 최소화 시켜주는 방향으로 튜닝하고 싶을경우. accuracy, roc-auc 같은 경우는 최대화 시켜주는 방향으로 maximize로 설정
study_lgb.optimize(objective, n_trials = 20) # optimize 최적함수 파라미터 찾는것. 즉 최적화

#과적합시 파라미터 튜닝
#min_chlid_weight 높이기, max_depth 낮추기. subsample, 트리 복잡도 낮추기, bagging_fraction[dafault = 1.0]: 트리가 커져서 과적합 하는것을 제어(==subsample) 샘플링 비율지정
trial = study_lgb.best_trial
trial_params = trial.params
print(f"best Trial:score{trial.value},\nparams{trial_params}")
print("Best Score:", study_lgb.best_value)
print("Best trial:", study_lgb.best_trial.params)

[I 2024-04-22 15:03:30,847] A new study created in memory with name: no-name-3af73f2d-fa04-4333-a506-6c382decc4a4
[I 2024-04-22 15:03:39,960] Trial 0 finished with value: 0.9825422470484408 and parameters: {'reg_alpha': 2.3141906091569712e-05, 'reg_lambda': 0.0018676852348266414, 'num_leaves': 163, 'max_depth': 12, 'learning_rate': 9.795848815655205e-06, 'n_estimators': 752, 'min_child_samples': 24, 'subsample': 0.8029815922829752, 'colsample_bytree': 0.5014665019375213}. Best is trial 0 with value: 0.9825422470484408.
[I 2024-04-22 15:03:49,701] Trial 1 finished with value: 0.9798770521021741 and parameters: {'reg_alpha': 2.6593110270785678e-06, 'reg_lambda': 0.06168238679950357, 'num_leaves': 245, 'max_depth': 3, 'learning_rate': 1.1834599907542846e-05, 'n_estimators': 2457, 'min_child_samples': 63, 'subsample': 0.7749529014248719, 'colsample_bytree': 0.5751256409023799}. Best is trial 1 with value: 0.9798770521021741.
[I 2024-04-22 15:04:03,366] Trial 2 finished with value: 0.982941

best Trial:score0.8899175942918016,
params{'reg_alpha': 8.597475168789943e-06, 'reg_lambda': 0.08528190763167148, 'num_leaves': 193, 'max_depth': 9, 'learning_rate': 0.007446401724927197, 'n_estimators': 158, 'min_child_samples': 63, 'subsample': 0.9980338359481813, 'colsample_bytree': 0.9687581255197978}
Best Score: 0.8899175942918016
Best trial: {'reg_alpha': 8.597475168789943e-06, 'reg_lambda': 0.08528190763167148, 'num_leaves': 193, 'max_depth': 9, 'learning_rate': 0.007446401724927197, 'n_estimators': 158, 'min_child_samples': 63, 'subsample': 0.9980338359481813, 'colsample_bytree': 0.9687581255197978}


In [79]:
#oof 4번학습 결과 도출
folds = KFold(n_splits = 4, shuffle = True, random_state = 37)
# validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
oof_preds = np.zeros((train_x.shape[0],3))# 학습된 모델의 valitation set 을 예측하여 결과 확률을 담을 array 생성// np.zeros : 0으로 채워진 [shape] 생성
# Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
test_preds = np.zeros((test_x.shape[0],3))
# train 데이터를 8:2 비율로 분할하여 학습과 검증에 사용
# 마찬가지로 데이터가 섞이지 않게, 검증 데이터는 train 데이터의 마지막 20%을 사용
final_lgb_model = lgb.LGBMClassifier(**trial_params)#optuna로 찾은 최적 하이퍼파라미터 적용
for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(train_x)):#train_x 데이터를 count idx, 학습 검증데이터 분류 idx
    print('##### interation', fold_idx, ' 시작')
    train_xxx = train_x.iloc[train_idx, :]#분류된 train_idx 값에 맞는 데이터 저장
    train_yyy = train_y.iloc[train_idx]
    valid_xxx = train_x.iloc[valid_idx, :]
    valid_yyy = train_y.iloc[valid_idx]
    final_lgb_model.fit(train_xxx, train_yyy, eval_set=[(train_xxx, train_yyy), (valid_xxx, valid_yyy)], verbose=200, early_stopping_rounds = 200)
    oof_preds[valid_idx] = final_lgb_model.predict_proba(valid_xxx, num_iteration=final_lgb_model.best_iteration_)
    test_preds += final_lgb_model.predict_proba(test_x, num_iteration = final_lgb_model.best_iteration_)/folds.n_splits


##### interation 0  시작
##### interation 1  시작
##### interation 2  시작
##### interation 3  시작


In [82]:
len(test_preds)

310

In [83]:
preds_result= []
for i in range(len(test_preds)):
     preds_result.append(np.argmax(test_preds[i]))
print(preds_result)
        

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [42]:
for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(train_x)):
    print('##### interation', fold_idx, ' 시작')
    print(fold_idx)
    print("-"*8)
    print(train_idx)
    print("-"*8)
    print(valid_idx)
    print("-"*8)
    train_xxx = train_x.iloc[train_idx, :]
    train_yyy = train_y.iloc[train_idx]
    valid_xxx = train_x.iloc[valid_idx, :]
    valid_yyy = train_y.iloc[valid_idx]
    print(train_xxx)
    print("-"*8)
    print(train_yyy)
    print("-"*8)
    print(valid_xxx)
    print("-"*8)
    print(valid_yyy)
    break

##### interation 0  시작
0
--------
[  0   2   3   5   6   7   8   9  10  12  13  15  16  17  19  20  22  23
  24  25  26  27  29  30  31  32  33  34  35  37  38  39  41  42  43  44
  45  46  47  48  50  51  52  53  54  55  57  58  59  61  62  64  66  68
  69  70  71  72  73  75  76  77  78  79  80  81  82  83  85  86  87  88
  89  90  91  92  93  95  97  98  99 101 104 106 108 109 110 111 112 113
 114 115 116 117 118 119 120 122 123 124 125 126 127 128 129 132 134 137
 139 140 141 142 143 144 145 149 150 151 154 155 156 158 159 160 161 163
 164 165 167 168 169 170 171 172 174 175 177 178 179 180 182 184 185 187
 189 190 192 193 194 195 196 197 198 199 202 203 204 205 206 210 211 212
 213 216 217 218 220 224 225 227 228 229 230 232 234 236 237 238 239 240
 241 242 243 244 245 246 247 248 249 250 251 254 256 257 258 260 261 263
 266 267 270 272 273 274 275 277 278 279 280 281 282 283 284 285 286 287
 288 289 290 292 293 294 295 296 297 298 299 300 301 302 304 305 306 307
 309 310 312 313 

In [53]:
t_final_lgb_model = lgb.LGBMClassifier(**trial_params)
for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(train_x)):
    print('##### interation', fold_idx, ' 시작')
    train_xxx = train_x.iloc[train_idx, :]
    train_yyy = train_y.iloc[train_idx]
    valid_xxx = train_x.iloc[valid_idx, :]
    valid_yyy = train_y.iloc[valid_idx]
    t_final_lgb_model.fit(train_xxx, train_yyy, eval_set=[(train_xxx, train_yyy), (valid_xxx, valid_yyy)], verbose=200, early_stopping_rounds = 200)
    oof_preds[valid_idx] = t_final_lgb_model.predict_proba(valid_xxx, num_iteration=t_final_lgb_model.best_iteration_)[:, 1]
    print(t_final_lgb_model.predict_proba(test_x, num_iteration = t_final_lgb_model.best_iteration_))
    print(t_final_lgb_model.predict_proba(test_x, num_iteration = t_final_lgb_model.best_iteration_))
    break

##### interation 0  시작
[0.83255436 0.83666684 0.8019348  0.63201248 0.58624359 0.68201169
 0.62165067 0.30330327 0.32467993 0.64123005 0.63635307 0.59599574
 0.60771704 0.37691876 0.39485602 0.826661   0.85343063 0.81755728
 0.85098689 0.83428647 0.81960911 0.84521346 0.61359693 0.77146012
 0.79655833 0.84150545 0.81673705 0.83629974 0.80327303 0.8224009
 0.81860049 0.85244134 0.81273802 0.84497658 0.82560955 0.3660201
 0.39438885 0.57898199 0.61179767 0.55572888 0.5690422  0.29130514
 0.26596941 0.77470618 0.83075333 0.84851288 0.81853198 0.78647691
 0.79630842 0.7938581  0.79877269 0.82616534 0.30584473 0.32158739
 0.33522347 0.86116227 0.89073679 0.82756028 0.83524448 0.82393812
 0.83791387 0.31092009 0.34869704 0.31910323 0.3782014  0.27301789
 0.27929609 0.81608127 0.84174011 0.85383118 0.8357586  0.57525582
 0.5011822  0.85295324 0.81854694 0.84790389 0.8281432  0.8241499
 0.75817193 0.51737637 0.76124223 0.73619084 0.81302501 0.83122711
 0.82987494 0.84705153 0.71545262 0.572747

In [55]:
print(t_final_lgb_model.predict_proba(test_x, num_iteration = t_final_lgb_model.best_iteration_)[:, 0])

[0.05411507 0.04938007 0.05988316 0.13715205 0.15249221 0.12424858
 0.16502395 0.40570078 0.43481567 0.20068098 0.20464687 0.1966598
 0.18095485 0.31589477 0.40792737 0.05739432 0.04508567 0.05709182
 0.04361975 0.0485954  0.05317191 0.0479116  0.22009067 0.09562452
 0.0844465  0.04922729 0.05701135 0.04852672 0.07706986 0.05515403
 0.07143347 0.04388444 0.07405256 0.04839394 0.05104945 0.40235755
 0.39540571 0.14468751 0.14321682 0.14961098 0.13983552 0.49352171
 0.45059857 0.05527655 0.05415849 0.04576009 0.05505075 0.09413076
 0.06237815 0.08386326 0.05622118 0.07360458 0.43510297 0.35105978
 0.46566548 0.05838379 0.0505256  0.05226755 0.05289257 0.05641569
 0.04929746 0.43404913 0.32191227 0.27632223 0.33016227 0.47474157
 0.48832588 0.04885713 0.04769969 0.04576787 0.04963272 0.13233908
 0.19002707 0.04795037 0.05766568 0.04526105 0.0513937  0.05587413
 0.07526075 0.21518073 0.07720602 0.06883835 0.05475552 0.05287872
 0.05415659 0.04536644 0.13548905 0.21988392 0.23234289 0.19017

# Predict

In [ ]:
pred = voting.predict(test_x)
sub['Y_Class'] = pred

sub.to_csv('./answer.csv', index=False)